In [1]:
import math
import random
import requests
import pandas as pd
import googlemaps
import numpy as np
import folium
import matplotlib.pyplot as plt
import koreanize_matplotlib
import urllib.request
import datetime
import time
from branca.colormap import LinearColormap
import pymysql
import pandas
import re
from glob import glob
import mysql.connector
import pandas as pd
import seaborn as sns
from haversine import haversine


import mysql.connector
import pandas as pd
import chardet
import altair as alt

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains  #동작을 하나로 묶기
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import json


from glob import glob #파일을 확인할 수 있는!
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from tqdm import tqdm_notebook
from urllib.request import urlopen, Request
import pandas as pd

import numpy as np
import googlemaps

/tmp/ipykernel_81659/1254186956.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
sub_od=pd.read_csv("../../data/eda_project/지하철/역별OD/지하철_역별OD_20240229.csv",encoding='cp949', index_col=0)

In [28]:
sub_od.index.unique()

Index([20240229], dtype='int64', name='기준일자')

In [5]:
cost=sub_od['총_승객수'].sum()
cost

6716595

In [7]:
remote = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user =  'eda_team',
    password = '123456',
    database = 'eda_project'
)

In [8]:
cursor = remote.cursor(buffered=True)
cursor.execute("select * from gihu_service1")

In [9]:
result = cursor.fetchall()
for row in result:
    print(row)

('가산디지털단지', '1호선', 'O', 'O', 'X', 'X', 'X')
('간석', '1호선', 'X', 'X', 'X', 'X', 'X')
('개봉', '1호선', 'O', 'O', 'X', 'X', 'X')
('관악', '1호선', 'X', 'X', 'X', 'X', 'X')
('광명', '1호선', 'X', 'X', 'X', 'X', 'X')
('광운대', '1호선', 'O', 'O', 'X', 'X', 'X')
('구로', '1호선', 'O', 'O', 'X', 'X', 'X')
('구일', '1호선', 'O', 'O', 'X', 'X', 'X')
('군포', '1호선', 'X', 'X', 'X', 'X', 'X')
('금정', '1호선', 'X', 'X', 'X', 'X', 'X')
('금천구청', '1호선', 'O', 'O', 'X', 'X', 'X')
('기능', '1호선', 'X', 'X', 'X', 'X', 'X')
('남영', '1호선', 'O', 'O', 'X', 'X', 'X')
('노량진', '1호선', 'O', 'O', 'X', 'X', 'X')
('녹양', '1호선', 'X', 'X', 'X', 'X', 'X')
('녹천', '1호선', 'O', 'O', 'X', 'X', 'X')
('당정', '1호선', 'X', 'X', 'X', 'X', 'X')
('대방', '1호선', 'O', 'O', 'X', 'X', 'X')
('덕계', '1호선', 'X', 'X', 'X', 'X', 'X')
('덕정', '1호선', 'X', 'X', 'X', 'X', 'X')
('도봉', '1호선', 'O', 'O', 'X', 'X', 'X')
('도봉산', '1호선', 'O', 'O', 'X', 'X', 'X')
('도원', '1호선', 'X', 'X', 'X', 'X', 'X')
('도화', '1호선', 'X', 'X', 'X', 'X', 'X')
('독산', '1호선', 'O', 'O', 'X', 'X', 'X')
('동대문', '1호선', 

In [10]:
gihu_ox = pd.DataFrame(result)
gihu_ox.head()

,0,1,2,3,4,5,6
0,가산디지털단지,1호선,O,O,X,X,X
1,간석,1호선,X,X,X,X,X
2,개봉,1호선,O,O,X,X,X
3,관악,1호선,X,X,X,X,X
4,광명,1호선,X,X,X,X,X


In [11]:
cursor.execute("SHOW COLUMNS FROM gihu_service1")

In [12]:
columns = [column[0] for column in cursor.fetchall()]

In [13]:
gihu_ox  = pd.DataFrame(result, columns=columns)

In [14]:
gihu_ox

,이름,노선,승차가능,하차가능,판매여부,충전여부,사용정지
0,가산디지털단지,1호선,O,O,X,X,X
1,간석,1호선,X,X,X,X,X
2,개봉,1호선,O,O,X,X,X
3,관악,1호선,X,X,X,X,X
4,광명,1호선,X,X,X,X,X
...,...,...,...,...,...,...,...
677,솔밭공원,우이신설,O,O,X,O,O
678,솔샘,우이신설,O,O,X,O,O
679,신설동,우이신설,O,O,X,O,O
680,정릉,우이신설,O,O,X,O,O


In [18]:
condition = (gihu_ox['승차가능'] == 'O') & (gihu_ox['하차가능'] == 'O')
filtered_rows = gihu_ox[condition]

In [20]:
filtered_rows.head()

,이름,노선,승차가능,하차가능,판매여부,충전여부,사용정지
0,가산디지털단지,1호선,O,O,X,X,X
2,개봉,1호선,O,O,X,X,X
5,광운대,1호선,O,O,X,X,X
6,구로,1호선,O,O,X,X,X
7,구일,1호선,O,O,X,X,X


In [21]:
condition1 = ~sub_od['승차_역'].isin(filtered_rows['이름'])
# sub_od의 하차역이 filtered_rows의 하차역에 포함되지 않는 행을 추출
condition2 = ~sub_od['하차_역'].isin(filtered_rows['이름'])

# 두 조건을 모두 만족하는 행을 추출하여 filtered_sub_od에 저장
filtered_sub_od = sub_od[condition1 & condition2]

In [25]:
cost_person=filtered_sub_od['총_승객수'].sum()
cost_person

598123

승차가능지역에서 고양시/부천시/용인시 하차 요금 

In [18]:
st_gihu_o = gihu_ox[gihu_ox['승차가능'] == 'O']

In [19]:
st_gihu_o

,이름,노선,승차가능,하차가능,판매여부,충전여부,사용정지
0,가산디지털단지,1호선,O,O,X,X,X
2,개봉,1호선,O,O,X,X,X
5,광운대,1호선,O,O,X,X,X
6,구로,1호선,O,O,X,X,X
7,구일,1호선,O,O,X,X,X
...,...,...,...,...,...,...,...
677,솔밭공원,우이신설,O,O,X,O,O
678,솔샘,우이신설,O,O,X,O,O
679,신설동,우이신설,O,O,X,O,O
680,정릉,우이신설,O,O,X,O,O


In [21]:
goyang = {
    '이름' : ['일산', '능곡', '원당','탄현','곡산','화정','삼송','백석','정발산','대곡','주엽','풍산'],
}
goyang_subway = pd.DataFrame(goyang)

In [ ]:
goyang_subway = pd.DataFrame(goyang)

In [26]:
goyang_off = sub_od[sub_od['하차_역'].isin(goyang_subway['이름'])]

In [27]:
goyang_off

,승차_호선,승차_역,하차_호선,하차_역,총_승객수,일반_승객수,어린이_승객수,청소년_승객수,대학생_승객수,경로_승객수,장애인_승객수,국가유공자_승객수,직원_승객수,외국인경로_승객수
기준일자,,,,,,,,,,,,,,
20240229,3호선,가락시장,경의중앙선,능곡,1,1,0,0,0,0,0,0,0,0
20240229,3호선,가락시장,3호선,대곡,2,2,0,0,0,0,0,0,0,0
20240229,3호선,가락시장,3호선,백석,14,10,0,0,0,3,1,0,0,0
20240229,8호선,가락시장,3호선,삼송,1,0,0,1,0,0,0,0,0,0
20240229,3호선,가락시장,3호선,삼송,26,20,0,0,0,5,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240229,9호선,흑석(중앙대입구),경의중앙선,일산,6,4,0,1,0,0,1,0,0,0
20240229,9호선,흑석(중앙대입구),3호선,정발산,1,0,0,0,0,1,0,0,0,0
20240229,9호선,흑석(중앙대입구),경의중앙선,탄현,4,2,0,0,0,2,0,0,0,0


In [126]:
goyang_on = sub_od[sub_od['승차_역'].isin(goyang_subway['이름'])]

In [127]:
goyang_on.head()

,승차_호선,승차_역,하차_호선,하차_역,총_승객수,일반_승객수,어린이_승객수,청소년_승객수,대학생_승객수,경로_승객수,장애인_승객수,국가유공자_승객수,직원_승객수,외국인경로_승객수
기준일자,,,,,,,,,,,,,,
20240229,경의중앙선,곡산,7호선,가산디지털단지,10,9,0,0,0,0,1,0,0,0
20240229,경의중앙선,곡산,9호선,가양,3,2,0,0,0,1,0,0,0,0
20240229,경의중앙선,곡산,경의중앙선,가좌,2,2,0,0,0,0,0,0,0,0
20240229,경의중앙선,곡산,2호선,강남,1,1,0,0,0,0,0,0,0,0
20240229,경의중앙선,곡산,8호선,강동구청,1,1,0,0,0,0,0,0,0,0


In [74]:
options = webdriver.ChromeOptions()

prefs = {'download.default_directory' : '../../data/17.gas_station_price',
         'download.prompt_for_download' : False}

options.add_experimental_option('prefs' , prefs)

url = "http://www.seoulmetro.co.kr/kr/cyberStation.do?menuIdx=538#wayInfo"
driver = webdriver.Chrome(service=Service("/home/min/dev_ws/EDA/driver/chromedriver"), #드라이버 경로는 절대경로로
                          options=options)

driver.get(url)

In [93]:
first_station = goyang_on.iloc[0]['승차_역']
first_station

'곡산'

In [94]:
end_station = goyang_on.iloc[0]['하차_역']
end_station

'가산디지털단지'

In [99]:
first_btn = driver.find_element(By.ID, "start-name")
first_btn.send_keys(first_station)
first_btn.send_keys(Keys.ENTER)

In [100]:
end_btn = driver.find_element(By.ID, "end-name")
end_btn.send_keys(end_station)
end_btn.send_keys(Keys.ENTER)

In [107]:
general_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[2]/span[1]').text
teen_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[3]/span[1]').text
kids_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[4]/span[1]').text

In [97]:
reset_btn = driver.find_element(By.XPATH, '//*[@id="searchReset"]/span')

In [98]:
reset_btn.click()

In [73]:
driver.close()

반복

In [128]:
general_fee = []
teen_fee = []
kids_fee = []

for idx in range(0,len(goyang_on)):
    if goyang_on.iloc[idx]['승차_역'] != goyang_on.iloc[idx]['하차_역']:
        first_station = goyang_on.iloc[idx]['승차_역']
        end_station = goyang_on.iloc[idx]['하차_역']
        first_btn = driver.find_element(By.ID, "start-name")
        first_btn.send_keys(first_station)
        first_btn.send_keys(Keys.ENTER)
        end_btn = driver.find_element(By.ID, "end-name")
        end_btn.send_keys(end_station)
        end_btn.send_keys(Keys.ENTER)
        time.sleep(3)
        general_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[2]/span[1]').text
        teen_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[3]/span[1]').text
        kids_fee=driver.find_element(By.XPATH, '/html/body/section/div/div[1]/article[2]/div[1]/p[4]/span[1]').text

        reset_btn = driver.find_element(By.XPATH, '//*[@id="searchReset"]/span')
        reset_btn.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=123.0.6312.58)
Stacktrace:
#0 0x5727d2d55793 <unknown>
#1 0x5727d2a49017 <unknown>
#2 0x5727d2a8e45f <unknown>
#3 0x5727d2a8c8a0 <unknown>
#4 0x5727d2ab63d2 <unknown>
#5 0x5727d2a88bd8 <unknown>
#6 0x5727d2ab659e <unknown>
#7 0x5727d2ad4dd9 <unknown>
#8 0x5727d2ab6173 <unknown>
#9 0x5727d2a872d3 <unknown>
#10 0x5727d2a87c9e <unknown>
#11 0x5727d2d198cb <unknown>
#12 0x5727d2d1d745 <unknown>
#13 0x5727d2d062e1 <unknown>
#14 0x5727d2d1e2d2 <unknown>
#15 0x5727d2cea17f <unknown>
#16 0x5727d2d43dc8 <unknown>
#17 0x5727d2d43fc3 <unknown>
#18 0x5727d2d54944 <unknown>
#19 0x746e5f494ac3 <unknown>
